## Variación espacio-temporal precipitación total

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/cerro_saroche
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [2]:
import pandas as pd

from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Conexión MongoDB 

In [4]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['meteorological', 'estimateSSTNino34', 'SSTNino34']

In [5]:
# Parque
park = 'cerro_saroche'#sys.argv[1]
id_point = 1#int(sys.argv[2])

y_output = 'ndvi_t'#sys.argv[3]
exogena = 'precipitation_ann_t'#sys.argv[4]

prediction_order = 2*24#int(sys.argv[5])# rango de prediccion
auto_order = 2*12#int(sys.argv[6]) # componente autoregresiva
exog_order = 2*12#int(sys.argv[7])# componente exogena qm
exog_delay = 0#int(sys.argv[8])# componente exogena dm

f_activation = 'sigmoid'#sys.argv[9]

# Parametros de modelos
patience = 15
epochs=500

# Descargando información

In [6]:
# Realizando consulta
meteorological = db.meteorological.find({"park":park, 'id_point':id_point})

# Generando pandas dataframe
data_pandas = pd.DataFrame([file for file in meteorological])
data_pandas['periodo'] = data_pandas.time.apply(lambda x: datetime.fromordinal(x))
data_pandas['mes_year'] =  data_pandas['periodo'].dt.strftime('%B-%Y')
#data_pandas.index = pd.to_datetime(data_pandas.periodo)

data_pandas.head()

,_id,id_point,park,time,elevacion_maxima,elevacion_media,elevacion_mediana,latitud,longitud,ndvi_maxima,ndvi_media,ndvi_mediana,precipitacion_mm,time_actualizacion,periodo,mes_year
0,633988a2eed0e0231b327c97,1,cerro_saroche,719163,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.913065,738430,1970-01-01,January-1970
1,633988a3eed0e0231b327d98,1,cerro_saroche,719194,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.081278,738430,1970-02-01,February-1970
2,633988a4eed0e0231b327e91,1,cerro_saroche,719222,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.413783,738430,1970-03-01,March-1970
3,633988a5eed0e0231b327f98,1,cerro_saroche,719253,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.895653,738430,1970-04-01,April-1970
4,633988a6eed0e0231b328114,1,cerro_saroche,719283,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,2.909450,738430,1970-05-01,May-1970


In [7]:
print(data_pandas[data_pandas.ndvi_media.notnull()].periodo.min())
print((data_pandas[data_pandas.ndvi_media.notnull()].periodo.max()))

2012-01-01 00:00:00
2022-05-01 00:00:00


In [8]:
list_interpolate = []

for id in data_pandas.sort_values('id_point',ascending=True).id_point.unique():

    pd_interpolate = data_pandas[[ 'id_point', 'latitud', 
                                'longitud', 'ndvi_media','periodo']]\
                                .query(f'id_point=={id}')\
                                .sort_values('periodo',ascending=True)

    pd_interpolate['ndvi_media'] = pd_interpolate['ndvi_media'].interpolate(method="linear")

    list_interpolate.append(pd_interpolate)

pd_ndvi = pd.concat(list_interpolate)

pd_ndvi.head()

,id_point,latitud,longitud,ndvi_media,periodo
0,1,10.31,-69.83,NaN,1970-01-01
1,1,10.31,-69.83,NaN,1970-02-01
2,1,10.31,-69.83,NaN,1970-03-01
3,1,10.31,-69.83,NaN,1970-04-01
4,1,10.31,-69.83,NaN,1970-05-01


## Cargando la data

In [9]:
pd_precipitacion = pd.read_pickle(f'./{park}/data/ann_precipitacion.pkl')
pd_precipitacion = pd_precipitacion[['park', 'periodo', 'year', 'month', 'id_point', 'latitud', 'longitud',
                                    'type', 'precipitacion_mm','elevacion_media', 'precipitacion_narx', 'prediction_ann']]

pd_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann
0,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,508.541046,0.355879,0.332924
1,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,508.541046,1.841747,0.461907
2,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,508.541046,1.174055,0.613882
3,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,508.541046,1.151622,0.768852
4,cerro_saroche,1995-06-01,1995,6,1,10.31,-69.83,training,0.573345,508.541046,0.850688,0.908978


In [10]:
pd_model = pd.merge(pd_precipitacion,pd_ndvi,
                    on=['periodo','id_point','latitud','longitud'],
                    how='left')

pd_model.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann,ndvi_media
0,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,508.541046,0.355879,0.332924,NaN
1,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,508.541046,1.841747,0.461907,NaN
2,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,508.541046,1.174055,0.613882,NaN
3,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,508.541046,1.151622,0.768852,NaN
4,cerro_saroche,1995-06-01,1995,6,1,10.31,-69.83,training,0.573345,508.541046,0.850688,0.908978,NaN


Aplicando transformación

In [11]:
# Transformacion
ndvi_transformacion = MinMaxScaler() #LogMinimax.create( pd_sst.oni.to_numpy() )
ndvi_transformacion.fit(pd_model[['prediction_ann','ndvi_media']])

pd_model[['precipitation_ann_t','ndvi_t']] = ndvi_transformacion.transform( pd_model[['prediction_ann','ndvi_media']] )
pd_model.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann,ndvi_media,precipitation_ann_t,ndvi_t
0,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,508.541046,0.355879,0.332924,NaN,0.106312,NaN
1,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,508.541046,1.841747,0.461907,NaN,0.208914,NaN
2,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,508.541046,1.174055,0.613882,NaN,0.329807,NaN
3,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,508.541046,1.151622,0.768852,NaN,0.453081,NaN
4,cerro_saroche,1995-06-01,1995,6,1,10.31,-69.83,training,0.573345,508.541046,0.850688,0.908978,NaN,0.564548,NaN


Directorio experimentos

In [12]:
DIR = f'./{park}/'
experimento = f'experiments/narx/ndvi/{id_point}'

try:
    os.mkdir(f'{DIR}{experimento}')
except:
    pass

# Ajustando modelo NARX

In [13]:
pd_model_id = pd_model[pd_model.id_point==id_point]
pd_model_id.index = pd.to_datetime(pd_model_id.periodo)
pd_model_id = pd_model_id[[y_output,exogena]].dropna()
pd_model_id.head()

,ndvi_t,precipitation_ann_t
periodo,,
2012-01-01,0.810476,0.021764
2012-02-01,0.603212,0.091347
2012-03-01,0.393684,0.186584
2012-04-01,0.745203,0.299522
2012-05-01,0.941786,0.415524


In [14]:
def split_data(pd_model_id,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output):
    """
    Funcion para dale estructura a los datos
    """

    x_data = []
    y_data = []

    min_index = max([exog_order+exog_delay,auto_order])
    index_split = pd_model_id[min_index:].index

    for t in range(len(index_split)):

        pd_to_split = pd_model_id[pd_model_id.index<=index_split[t]][-min_index-1:]

        exogen_values = pd_to_split[(pd_to_split.shape[0]-exog_delay-exog_order):(pd_to_split.shape[0]-exog_delay)][[exogena]].values.reshape(-1)
        auto_values = pd_to_split[-auto_order-1:][[y_output]].values.reshape(-1)

        x_data.append(np.concatenate([exogen_values, auto_values[:-1]],axis=None))
        y_data.append(auto_values[-1])
        
    x_data = np.array(x_data)
    y_data = np.array(y_data)

    return x_data, y_data

In [15]:
x_data, y_data = split_data(pd_model_id,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output)

Entrenamiento y validación

In [16]:
x_train = x_data[:-prediction_order]
x_vasl = x_data[-prediction_order:]

y_train = y_data[:-prediction_order]
y_vasl = y_data[-prediction_order:]

Modelo NARX

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [18]:
# Metrícas
mae = keras.metrics.MeanAbsoluteError()
rmse = keras.metrics.RootMeanSquaredError()

confi = {'Input':{'batch_size':None,
                'name':'input',
                'dtype':None,
                'sparse':None,
                'tensor':None,
                'ragged':None,
                'type_spec':None},
        'Dense':{'use_bias':True,
                'kernel_regularizer':None,
                'bias_regularizer':None,
                'activity_regularizer':None,
                'kernel_constraint':None,
                'bias_constraint':None
                }
        }

In [19]:
total = int(2*x_train.shape[-1]/3)
n_neurons = [int(3*total/6), int(2*total/6), int(1*total/6), 1]

activation = len(n_neurons)*[f_activation]
kernel_initializer = 'lecun_normal'
bias_initializer = 'zeros'

In [20]:
# Modelo
model = keras.models.Sequential()

# Entradas
model.add(keras.layers.Input(shape=(x_train.shape[-1],),
                                    batch_size = confi.get('Input').get('batch_size'),
                                    name = confi.get('Input').get('name'),
                                    dtype = confi.get('Input').get('dtype'),
                                    sparse = confi.get('Input').get('sparse'),
                                    tensor = confi.get('Input').get('tensor'),
                                    ragged = confi.get('Input').get('ragged'),
                                    type_spec = confi.get('Input').get('type_spec')
                                    ))

model.add(keras.layers.Dense(   units=n_neurons[0],
                                activation=activation[0],
                                use_bias = confi.get('Dense').get('use_bias'),
                                kernel_initializer=kernel_initializer,
                                bias_initializer=bias_initializer,
                                kernel_regularizer = confi.get('Dense').get('kernel_regularizer'),
                                bias_regularizer = confi.get('Dense').get('bias_regularizer'),
                                activity_regularizer = confi.get('Dense').get('activity_regularizer'),
                                kernel_constraint = confi.get('Dense').get('kernel_constraint'),
                                bias_constraint = confi.get('Dense').get('bias_constraint')
                                ))
                                
model.add(keras.layers.Dropout(0.1))

# Hidden Leyers
if len(n_neurons)>1:
    for index in list( range(1, len(n_neurons)) ):

        model.add(keras.layers.Dense(   units=n_neurons[index],
                                        activation=activation[index],
                                        use_bias = confi.get('Dense').get('use_bias'),
                                        kernel_initializer=kernel_initializer,
                                        bias_initializer=bias_initializer,
                                        kernel_regularizer = confi.get('Dense').get('kernel_regularizer'),
                                        bias_regularizer = confi.get('Dense').get('bias_regularizer'),
                                        activity_regularizer = confi.get('Dense').get('activity_regularizer'),
                                        kernel_constraint = confi.get('Dense').get('kernel_constraint'),
                                        bias_constraint = confi.get('Dense').get('bias_constraint')
                                        ))
                                        
        # model.add(keras.layers.Dropout(0.001))
        # print()

# Out
model.add(keras.layers.Dense(   units=1,
                                activation='linear',
                                kernel_initializer=kernel_initializer,
                                bias_initializer=bias_initializer
                                ))
                                

model.compile(loss='mean_squared_error', optimizer='adam', metrics=[mae,rmse]) 

In [21]:
callback = keras.callbacks.EarlyStopping(
                                            monitor="loss",
                                            min_delta=0,
                                            patience=patience,
                                            verbose=0,
                                            mode="min",
                                            baseline=None,
                                            restore_best_weights=False,
                                        )

Entrenamiento

In [22]:
history = model.fit(x=x_train,
                    y=y_train,
                    epochs=epochs,
                    batch_size=1,
                    verbose=0,
                    workers=2,
                    callbacks=[callback])

print(f'Total epocas:{len(history.epoch)}')

Total epocas:214


Evaluación entrenamiento

In [23]:
# make predictions
trainPredict = model.predict(x_train, verbose=0).reshape(-1)
testPredict = model.predict(x_vasl, verbose=0).reshape(-1)

In [24]:
# Data de test
trainind_pd = pd.DataFrame(trainPredict,
                            index = pd_model_id[-(trainPredict.shape[0]+prediction_order):-(prediction_order)].index,
                            columns=['prediction']
                            )

trainind_pd[y_output] = y_train.reshape(-1)
trainind_pd['type'] = 'training'
trainind_pd['precipitation_ann_t'] = np.nan

trainind_pd['id_point'] = id_point

trainind_pd[['prediction_ann','ndvi_prediction']] = ndvi_transformacion.inverse_transform(trainind_pd[['precipitation_ann_t','prediction']])
trainind_pd[['prediction_ann','ndvi_media']] = ndvi_transformacion.inverse_transform(trainind_pd[['precipitation_ann_t',y_output]])

trainind_pd = trainind_pd.reset_index(drop=False)[['id_point', 'periodo','type','ndvi_prediction','ndvi_media']]

In [25]:
# Validacion entrenamiento
trainig_metrics = metrics(observado=trainind_pd.ndvi_media,
                          prediccion=trainind_pd.ndvi_prediction)

Evaluación validación

In [26]:
# Data de test
validation_pd = pd.DataFrame(testPredict,
                            index = pd_model_id[-prediction_order:].index,
                            columns=['prediction']
                            )

validation_pd[y_output] = y_vasl.reshape(-1)
validation_pd['type'] = 'validation'
validation_pd['precipitation_ann_t'] = np.nan

validation_pd['id_point'] = id_point

validation_pd[['prediction_ann','ndvi_prediction']] = ndvi_transformacion.inverse_transform(validation_pd[['precipitation_ann_t','prediction']])
validation_pd[['prediction_ann','ndvi_media']] = ndvi_transformacion.inverse_transform(validation_pd[['precipitation_ann_t',y_output]])

validation_pd = validation_pd.reset_index(drop=False)[['id_point', 'periodo','type','ndvi_prediction','ndvi_media']]

In [27]:
# Validacion entrenamiento
validation_metrics = metrics(observado=validation_pd.ndvi_media,
                          prediccion=validation_pd.ndvi_prediction)

Sección Test

In [28]:
def predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output):
    """
    Funcion para predecir a un paso
    """
    
    data_proces = pd.concat([data_predict,data_exogena[list(data_predict)]])
    data_proces['type'] = 'data_in'

    date_min = data_proces[data_proces[y_output].isnull()].index.min()
    date_max = data_proces[data_proces[y_output].isnull()].index.max()

    date = date_min
    while date <= date_max:
        x_data_test, y_data_test = split_data(data_proces[data_proces.index<=date],
                                                exog_order,
                                                auto_order,
                                                exog_delay,
                                                prediction_order,
                                                exogena,y_output)

        predit = model.predict(x_data_test[-1].reshape(1, x_data_test.shape[1]), verbose=0).reshape(-1)
        data_proces.loc[(data_proces.index==date),y_output]=predit

        date = data_proces[data_proces[y_output].isnull()].index.min()

    return data_proces[data_proces.index>=date_min]

In [29]:
data_exogena = pd_model_id[-prediction_order:][[exogena]]
data_exogena[y_output] = np.nan
data_predict = pd_model_id[pd_model_id.index < data_exogena.index.min()][[y_output,exogena]]

In [30]:
pd_test = predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output)

In [31]:
pd_test = predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output)

pd_test = pd_test.rename(columns={y_output:'prediction'})
pd_test['type'] = 'test'

pd_test[y_output] = pd_model_id[pd_model_id.index > trainind_pd.periodo.max()][y_output]

pd_test['id_point'] = id_point

pd_test[['prediction_ann','ndvi_prediction']] = ndvi_transformacion.inverse_transform(pd_test[['precipitation_ann_t','prediction']])
pd_test[['prediction_ann','ndvi_media']] = ndvi_transformacion.inverse_transform(pd_test[['precipitation_ann_t',y_output]])

pd_test = pd_test.reset_index(drop=False)[['id_point', 'periodo','type','ndvi_prediction','ndvi_media']]

In [32]:
# Validacion entrenamiento
test_metrics = metrics(observado=pd_test.ndvi_media,
                        prediccion=pd_test.ndvi_prediction)

Resultados del modelo

In [33]:
# Resultados del modelo
dict_metrics = {'epocas':[len(history.epoch)],
                'prediction_order':[prediction_order],
                'auto_order':[auto_order],
                'exog_order':[exog_order],
                'exog_delay':[exog_delay],
                'activation':[activation[0]],
                'id_point':[id_point],
                'n_neurons':str(n_neurons),
                'capas':[len(n_neurons)],
                'training_mse':[trainig_metrics["mse"]],
                'training_rmse':[trainig_metrics["rmse"]],
                'training_mae':[trainig_metrics["mae"]],
                'trainig_mape':[trainig_metrics['mape']],
                'trainig_r':[trainig_metrics['r2']],
                'validation_mse':[validation_metrics["mse"]],
                'validation_rmse':[validation_metrics["rmse"]],
                'validation_mae':[validation_metrics["mae"]],
                'validation_mape':[validation_metrics['mape']],
                'validation_r':[validation_metrics['r2']],
                'test_mse':[test_metrics["mse"]],
                'test_rmse':[test_metrics["rmse"]],
                'test_mae':[test_metrics["mae"]],
                'test_mape':[test_metrics['mape']],
                'test_r':[test_metrics['r2']]
                }

experimento_pd = pd.DataFrame.from_dict(dict_metrics)

Pronóstico

In [34]:
data_predict = pd_model_id[[y_output,exogena]]

data_exogena = pd_model[(pd_model.periodo > data_predict.index.max()) & (pd_model.id_point==id_point)][[exogena,'periodo']]
data_exogena.index = pd.to_datetime(data_exogena.periodo)
data_exogena[y_output] = np.nan
data_exogena = data_exogena.sort_index()[[exogena,y_output]]

pd_prediction = predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order, exogena, y_output)
pd_prediction = pd_prediction.rename(columns={y_output:'prediction'})
pd_prediction['type'] = 'prediction'
pd_prediction['id_point'] = id_point


pd_prediction[['prediction_ann','ndvi_prediction']] = ndvi_transformacion.inverse_transform(pd_prediction[['precipitation_ann_t','prediction']])
pd_prediction['ndvi_media'] = np.nan

pd_prediction = pd_prediction.reset_index(drop=False)[['id_point', 'periodo','type','ndvi_prediction','ndvi_media']]

In [35]:
# Uniendo informacion
pd_summary = pd.concat([trainind_pd[list(pd_prediction)], 
                        pd_test[list(pd_prediction)], 
                        validation_pd[list(pd_prediction)], 
                        pd_prediction[list(pd_prediction)]
                        ])

Logica de guardado

In [36]:
import pickle

if os.listdir(f'{DIR}{experimento}') == []:

    # Modelo
    model.save(f'{DIR}{experimento}/model.h5')

    # Pesos
    model.save_weights(f'{DIR}{experimento}/weights.h5')

    # History
    with open(f'{DIR}{experimento}/history.pkl', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    # guardando resultados
    pd_summary.to_pickle(f'{DIR}{experimento}/predicciones.pkl')

else:
    files = [x for x in os.listdir(f'{DIR}{experimento}') if x.find('summary')!=-1 ]
    total_summary = pd.concat([pd.read_csv(f'{DIR}{experimento}/{file}') for file in files])
    print( f"Actual: {validation_metrics['r2']}; Best Model: {total_summary.validation_r.max()}" )

    if validation_metrics['r2'] > total_summary.validation_r.max(): 

        # Modelo
        model.save(f'{DIR}{experimento}/model.h5')

        # Pesos
        model.save_weights(f'{DIR}{experimento}/weights.h5')

        # History
        with open(f'{DIR}{experimento}/history.pkl', 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
        
        # guardando resultados
        pd_summary.to_pickle(f'{DIR}{experimento}/predicciones.pkl')

Actual: 0.2618402684383693; Best Model: 0.6129741623134988


In [37]:
experi = f'{DIR}{experimento}/{id_point}_{len(n_neurons)}_{activation[0]}_{prediction_order}_{auto_order}_{exog_order}_{exog_delay}'
experimento_pd.to_csv(f'{experi}_summary.csv',index=False)